# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Group Project by Dolphin Sharma and Charles Crocicchia

## Data Preprocessing

In [3]:
# Import necessary libraries
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Load & Merge Data

Loaded the relevant datasets for soybean data, corn data, and economic data (e.g., employment, income, poverty levels). These datasets were merged based on common keys like state and year to create a comprehensive dataset for analysis.

In [4]:
# Function to load and merge all data files within a directory
def load_all_data_from_directory(directory_path):
    """
    Loads all CSV files from a specified directory and returns a list of dataframes.

    Parameters:
    directory_path (str): The path to the directory containing CSV files.

    Returns:
    list: A list of pandas DataFrames, each representing the data from one CSV file.
    """
    all_data = []
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(directory_path, file_name)
            data = pd.read_csv(file_path)
            all_data.append(data)
    return all_data

In [5]:
# Load and merge corn data
def merge_corn_data(corn_dir):
    """
    Merges all corn data CSV files from the specified directory into a single DataFrame.

    Parameters:
    corn_dir (str): The path to the directory containing corn-related CSV files.

    Returns:
    pandas.DataFrame: A DataFrame resulting from merging the corn data files.
    """
    corn_data = load_all_data_from_directory(corn_dir)
    corn_merged = pd.concat(corn_data, axis=1, join='inner')
    return corn_merged

In [6]:
# Load and merge soybean data
def merge_soybean_data(soybean_dir):
    """
    Merges all soybean data CSV files from the specified directory into a single DataFrame.

    Parameters:
    soybean_dir (str): The path to the directory containing soybean-related CSV files.

    Returns:
    pandas.DataFrame: A DataFrame resulting from merging the soybean data files.
    """
    soybean_data = load_all_data_from_directory(soybean_dir)
    soybean_merged = pd.concat(soybean_data, axis=1, join='inner')
    return soybean_merged

In [7]:
# Load and merge economic data
def merge_economic_data(economic_dir):
    """
    Merges all economic data CSV files from the specified directory into a single DataFrame.

    Parameters:
    economic_dir (str): The path to the directory containing economic-related CSV files.

    Returns:
    pandas.DataFrame: A DataFrame resulting from merging the economic data files.
    """
    economic_data = load_all_data_from_directory(economic_dir)
    economic_merged = pd.concat(economic_data, axis=1, join='inner')
    return economic_merged

In [8]:
# Directories for the data
corn_dir = r'../data/clean_data/corn_data/'
soybean_dir = r'../data/clean_data/soybean_data/'
economic_dir = r'../data/clean_data/economic_data/'

In [9]:
# Merge all data
corn_merged = merge_corn_data(corn_dir)
soybean_merged = merge_soybean_data(soybean_dir)
economic_merged = merge_economic_data(economic_dir)

In [10]:
# Merge corn and soybean data with economic data
corn_soybean_merged = pd.concat([corn_merged, soybean_merged], axis=1, join='inner')
final_merged_data = pd.concat([corn_soybean_merged, economic_merged], axis=1, join='inner')

## Quick Data Cleanup After Merge

Removed unnecessary columns 'Unnamed', removed duplicated column(s), and removed numerical suffices from column names.

In [11]:
# Cleaning the data by removing unnecessary columns
final_merged_data_clean = final_merged_data.loc[:, ~final_merged_data.columns.str.contains('Unnamed')]

In [12]:
# Remove duplicate columns (like 'State.1')
final_merged_data_clean = final_merged_data_clean.loc[:, ~final_merged_data_clean.columns.duplicated()]

In [13]:
# Remove numerical suffixes from column names
final_merged_data_clean.columns = final_merged_data_clean.columns.str.replace(r'\.\d+$', '', regex=True)

## Renaming/Categorizing Columns 

Columns are renamed or categorized based on the type of data they represented (e.g., corn, soybean, or economic indicators) to make them easier to work with and identify during the modeling process.

In [14]:
# Renaming and categorizing columns
final_merged_data_clean.rename(columns={
    'Value': 'Corn_Acres_Planted',
    '2008': 'Corn_Cash_Receipts',
    '2009': 'Corn_Production_Bushels',
    '2010': 'Corn_Production_Dollars',
    '2011': 'Corn_Yield_Bushels',
    '2012': 'Soybean_Acres_Planted',
    '2013': 'Soybean_Cash_Receipts',
    '2014': 'Soybean_Production_Bushels',
    '2015': 'Soybean_Production_Dollars',
    '2016': 'Soybean_Yield_Bushels',
    '2017': 'Employment_Rural',
    '2018': 'Personal_Income_Rural',
    '2019': 'Poverty_Levels',
    '2020': 'Economic_Indicator_2020',
    '2021': 'Economic_Indicator_2021',
    '2022': 'Economic_Indicator_2022',
    '2023': 'Economic_Indicator_2023',
    '2000': 'Historical_Economic_2000',
    '2001': 'Historical_Economic_2001',
    '2002': 'Historical_Economic_2002',
    '2003': 'Historical_Economic_2003',
    '2004': 'Historical_Economic_2004',
    '2005': 'Historical_Economic_2005',
    '2006': 'Historical_Economic_2006',
    '2007': 'Historical_Economic_2007'
}, inplace=True)

## Handling Missing Values

Missing values in the dataset are addressed by dropping rows or columns with too many missing entries. Alternatively, imputation strategies (filling with median) are used for columns with fewer missing values.

In [15]:
# Function to handle missing values
def impute_missing_values(data, columns_to_impute):
    """
    Imputes missing values in the specified columns using the mean of the columns.

    Parameters:
    data (pandas.DataFrame): The input DataFrame containing data with missing values.
    columns (list): A list of column names for which missing values need to be imputed.

    Returns:
    pandas.DataFrame: The DataFrame with missing values imputed.
    """
    imputer = SimpleImputer(strategy='median')
    data[columns_to_impute] = imputer.fit_transform(data[columns_to_impute])
    return data

## Dealing with Collinearity

The Variance Inflation Factor (VIF) is calculated to check for multicollinearity among the features. Features with high VIF values are either dropped or combined to reduce collinearity, which could negatively impact the model performance.

In [16]:
# Function to calculate VIF (Variance Inflation Factor)
def calculate_vif(data, features):
    """
    Calculates the Variance Inflation Factor (VIF) for a given set of features in the dataset.

    Parameters:
    data (pandas.DataFrame): The input DataFrame containing the feature data.
    features (list): A list of feature column names for which VIF will be calculated.

    Returns:
    pandas.DataFrame: A DataFrame containing each feature and its corresponding VIF score.
    """
    vif_data = pd.DataFrame()
    vif_data["Feature"] = features
    vif_data["VIF"] = [variance_inflation_factor(data[features].values, i) for i in range(len(features))]
    return vif_data

## Functions for Preprocessing Data by Model

### Scaling in Linear Regression

Certain columns with large values (e.g., corn cash receipts, soybean production) are scaled to ensure that all features were on a similar scale. This step is important for models sensitive to feature scaling (e.g., linear regression).

In [17]:
# Preprocessing function for Linear Regression (with scaling for all numerica columns and VIF calculation)
def preprocess_for_linear_regression(data):
    """
    Preprocesses data for linear regression, including imputing missing values and scaling the features.

    Parameters:
    data (pandas.DataFrame): The input DataFrame to preprocess.

    Returns:
    pandas.DataFrame: A preprocessed DataFrame suitable for linear regression.
    """
    # Imputing missing values
    columns_to_impute = [
        'Corn_Acres_Planted', 'Soybean_Acres_Planted', 'Employment_Rural',
        'Corn_Cash_Receipts', 'Corn_Production_Bushels', 'Corn_Production_Dollars', 'Corn_Yield_Bushels',
        'Soybean_Cash_Receipts', 'Soybean_Production_Bushels', 'Soybean_Production_Dollars', 'Soybean_Yield_Bushels'
    ]
    data = impute_missing_values(data, columns_to_impute)
    
    # Scaling the data (including all numeric columns that should be scaled)
    scaler = StandardScaler()
    data[columns_to_impute] = scaler.fit_transform(data[columns_to_impute])
    
    # Calculate VIF for linear regression (only including primary features to avoid overfitting)
    features = ['Corn_Acres_Planted', 'Soybean_Acres_Planted', 'Employment_Rural']
    vif_data = calculate_vif(data, features)
    print("Variance Inflation Factors:\n", vif_data)
    
    return data

In [18]:
# Preprocessing function for Decision Tree (no scaling needed)
def preprocess_for_decision_tree(data):
        """
    Preprocesses data for decision tree models, focusing on imputing missing values.

    Parameters:
    data (pandas.DataFrame): The input DataFrame to preprocess.

    Returns:
    pandas.DataFrame: A preprocessed DataFrame suitable for decision tree models.
    """
    return impute_missing_values(data, ['Corn_Acres_Planted', 'Soybean_Acres_Planted', 'Employment_Rural'])

In [19]:
# Preprocessing function for Random Forest (no scaling needed)
def preprocess_for_random_forest(data):
    """
    Preprocesses data for random forest models, focusing on imputing missing values.

    Parameters:
    data (pandas.DataFrame): The input DataFrame to preprocess.

    Returns:
    pandas.DataFrame: A preprocessed DataFrame suitable for random forest models.
    """
    return impute_missing_values(data, ['Corn_Acres_Planted', 'Soybean_Acres_Planted', 'Employment_Rural'])

In [20]:
# Applying preprocessing to the cleaned dataset
linear_regression_data = preprocess_for_linear_regression(final_merged_data_clean.copy())
decision_tree_data = preprocess_for_decision_tree(final_merged_data_clean.copy())
random_forest_data = preprocess_for_random_forest(final_merged_data_clean.copy())

Variance Inflation Factors:
                  Feature         VIF
0     Corn_Acres_Planted    1.237742
1  Soybean_Acres_Planted  158.022893
2       Employment_Rural  160.861759


In [21]:
# Saving the preprocessed data for each model
linear_regression_data.to_csv(r'../data/processed_data/linear_regression_data.csv', index=False)
decision_tree_data.to_csv(r'../data/processed_data/decision_tree_data.csv', index=False)
random_forest_data.to_csv(r'../data/processed_data/random_forest_data.csv', index=False)